In [1]:
import tensorflow as tf

from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2

# from spektral.layers import GraphConv
# GRaphConv is deprecated, use GCNConv or GCSConv instead
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.utils import normalized_laplacian
from spektral.layers import GCSConv  # as GraphConv
from spektral.layers import GINConv # as GraphConv
from spektral.layers import GCNConv  # as GraphConv

from spektral.utils.convolution import gcn_filter  # For GCNConv
from spektral.utils.convolution import normalized_adjacency  # For GCSConv

import numpy as np
import scipy.sparse
import pandas as pd
import pathlib
import os
import json

from sklearn.model_selection import train_test_split

from utils import *

In [2]:
# Parameters
l2_reg = 5e-4  # Regularization rate for l2
learning_rate = 1e-3  # Learning rate for SGD
batch_size = 32  # Batch size
epochs = 5  # Number of training epochs
es_patience = 200  # Patience fot early stopping

In [3]:
current_path = pathlib.Path().absolute()
path = current_path.parent

In [4]:
angry_path = path / 'angry_meshpoints'
disgusted_path = path / 'disgusted_meshpoints'
happy_path = path / 'happy_meshpoints'
neutral_path = path / 'neutral_meshpoints'
sad_path = path / 'sad_meshpoints'
surprised_path = path / 'surprised_meshpoints'

path_list = [angry_path, disgusted_path, happy_path, neutral_path, sad_path, surprised_path]

In [46]:
def extrai_dist(path_list, meshs_list, limit=0.1):
    for path in path_list:
        quant_files = len(os.listdir(path))
        count = 0
        for file in os.listdir(path):
            if (count / quant_files) > limit:
                break
            file_path = path / file
            count += 1
            with open(file_path, 'r') as f:
                data = json.load(f)
                meshs_list[path_list.index(path)].append(data)

In [57]:
meshs_surprised = []
meshs_disgusted = []
meshs_happy = []
meshs_neutral = []
meshs_sad = []
meshs_angry = []

meshs_list = [meshs_angry, meshs_disgusted, meshs_happy, meshs_neutral, meshs_sad, meshs_surprised]

extrai_dist(path_list, meshs_list, 0.1)

print(len(meshs_angry))
print(len(meshs_disgusted))
print(len(meshs_happy))
print(len(meshs_neutral))
print(len(meshs_sad))
print(len(meshs_surprised))

446
161
810
687
742
402


In [58]:
# print(meshs_list)

In [59]:
print(len(meshs_list))

6


In [60]:
# target list
target_list = []
for i in range(6):
    target_list.append(np.full(len(meshs_list[i]), i))

In [61]:
# print(target_list)

In [62]:
# print the unique target labels
print(np.unique(np.concatenate(target_list)))

[0 1 2 3 4 5]


In [63]:
# concatena meshs_list em uma lista só
meshs_list_concat = np.concatenate(meshs_list)
target_list_concat = np.concatenate(target_list)

print(len(meshs_list_concat))

3248


# Iniciando a construção do primeiro GINConv

In [64]:
from sklearn.model_selection import train_test_split


# Split the dataset into train and test sets, shuffling the data
X_train, X_test, y_train, y_test = train_test_split(meshs_list_concat, target_list_concat, test_size=0.1, shuffle=True, random_state=42)

# Split the train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, random_state=42)

In [65]:
# import concat from tensorflow
from tensorflow import concat

# tensor_mesh = concat(meshs_list, axis=0)
# target_tensor = concat(target_list, axis=0)

X__train_tensor = concat(X_train, axis=0)
y__train_tensor = concat(y_train, axis=0)

X__val_tensor = concat(X_val, axis=0)
y__val_tensor = concat(y_val, axis=0)

X__test_tensor = concat(X_test, axis=0)
y__test_tensor = concat(y_test, axis=0)

In [66]:
for target_tensor in [y__train_tensor, y__val_tensor, y__test_tensor]:
    unique_values = np.unique(target_tensor.numpy())
    print("Unique values in target tensor:")
    print(unique_values)
    print()

Unique values in target tensor:
[0 1 2 3 4 5]

Unique values in target tensor:
[0 1 2 3 4 5]

Unique values in target tensor:
[0 1 2 3 4 5]



In [67]:
# tensor_mesh.shape
X__train_tensor.shape

TensorShape([2630, 478, 2])

In [68]:
# target_tensor.shape
y__train_tensor.shape

TensorShape([2630])

In [69]:
X__val_tensor.shape

TensorShape([293, 478, 2])

In [70]:
y__val_tensor.shape

TensorShape([293])

In [71]:
n_out = 6  # Number of classes
N = X__train_tensor.shape[-2]  # Number of nodes in the graphs
F = X__train_tensor.shape[-1]  # Original feature dimensionality

In [72]:
N, F

(478, 2)

In [73]:
adj = tf.sparse.from_dense(tf.convert_to_tensor(get_mediapipe_adjacency_matrix()))

X_in = Input(shape=(N, F))

graph_conv_1 = GINConv(32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True)([X_in, adj])
graph_conv_2 = GINConv(32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True)([graph_conv_1, adj])
flatten = Flatten()(graph_conv_2)
fc = Dense(512, activation="relu")(flatten)
output = Dense(n_out, activation="softmax")(fc)

In [74]:
# Build model
model = Model(inputs=X_in, outputs=output)
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"]
)

In [75]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 478, 2)]          0         
                                                                 
 gin_conv (GINConv)          (None, 478, 32)           97        
                                                                 
 gin_conv_1 (GINConv)        (None, 478, 32)           1057      
                                                                 
 flatten (Flatten)           (None, 15296)             0         
                                                                 
 dense (Dense)               (None, 512)               7832064   
                                                                 
 dense_1 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 7836296 (29.89 MB)
Trainable params: 7836296 (2

In [76]:
# o tensor_mesh é um tensor de dimensão 3
# o sparse_matrix é um tensor de dimensão 2
# o target_list é um tensor de dimensão 1

# Train model
model.fit(
    X__train_tensor,
    y__train_tensor,
    batch_size=batch_size,
    validation_data=(X__val_tensor, y__val_tensor),
    epochs=epochs,
    callbacks=[EarlyStopping(patience=es_patience)],
)


Epoch 1/5
83/83 [==============================] - 5s 50ms/step - loss: 1993.0509 - acc: 0.1981 - val_loss: 136.0345 - val_acc: 0.2594
Epoch 2/5
83/83 [==============================] - 4s 43ms/step - loss: 113.4394 - acc: 0.2152 - val_loss: 32.9159 - val_acc: 0.1160
Epoch 3/5
83/83 [==============================] - 4s 44ms/step - loss: 41.8737 - acc: 0.2384 - val_loss: 17.1881 - val_acc: 0.3413
Epoch 4/5
83/83 [==============================] - 4s 44ms/step - loss: 17.1835 - acc: 0.2673 - val_loss: 11.3134 - val_acc: 0.3038
Epoch 5/5
83/83 [==============================] - 4s 44ms/step - loss: 13.6699 - acc: 0.2681 - val_loss: 3.8753 - val_acc: 0.2560


# A celua abaixo foi para verificar se não tinha inconsistencia nos inputs/outputs

In [78]:
class0startsat = None
class1startsat = None
class2startsat = None
class3startsat = None
class4startsat = None
class5startsat = None

for i in range(len(y__val_tensor)):
    if y__val_tensor[i] == 0:
        class0startsat = i
        break

for i in range(len(y__val_tensor)):
    if y__val_tensor[i] == 1:
        class1startsat = i
        break

for i in range(len(y__val_tensor)):
    if y__val_tensor[i] == 2:
        class2startsat = i
        break

for i in range(len(y__val_tensor)):
    if y__val_tensor[i] == 3:
        class3startsat = i
        break

for i in range(len(y__val_tensor)):
    if y__val_tensor[i] == 4:
        class4startsat = i
        break

for i in range(len(y__val_tensor)):
    if y__val_tensor[i] == 5:
        class5startsat = i
        break

for i in [class0startsat, class1startsat, class2startsat, class3startsat, class4startsat, class5startsat]:
    if i is not None:
        input_id = i
        real_class = y__val_tensor[i].numpy()
        predicted_class = np.argmax(model.predict(X__val_tensor[i:i+1]))
        print("Input ID:", input_id)
        print("Real Class:", real_class)
        print("Predicted Class:", predicted_class)
        print()


1/1 [==============================] - 0s 16ms/step
Input ID: 6
Real Class: 0
Predicted Class: 3

1/1 [==============================] - 0s 15ms/step
Input ID: 5
Real Class: 1
Predicted Class: 3

1/1 [==============================] - 0s 15ms/step
Input ID: 4
Real Class: 2
Predicted Class: 3

1/1 [==============================] - 0s 15ms/step
Input ID: 1
Real Class: 3
Predicted Class: 1

1/1 [==============================] - 0s 11ms/step
Input ID: 0
Real Class: 4
Predicted Class: 3

1/1 [==============================] - 0s 15ms/step
Input ID: 3
Real Class: 5
Predicted Class: 1

